- grid search
 
but it is difficult in multidimensional...


then you can do a random walk search to explore the space of hyperparameters


then....

cross validation (k fold): compute the STANDARD DEVIATION associated to the accuracy, across the folds. then you can make a meaningful comparison.



AUGMENTATION:

pretend you dont have the function to generate the data. you need to create artificial ones.

small displacement and give it the same class as the original one. 
if done with moderation, it increases accuracy of the nn. if you do it too much, you risk. 

an example of augmentation is the rotation of an image with different angles (always the same image e.g. a tumor, or a cat... or whatever) but for the nn is a new sample 

#### Imports

In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras_tuner 
from keras.optimizers import Adam, RMSprop, Nadam

#import matplotlib as mpl
import matplotlib.pyplot as plt
# default font
plt.rcParams['font.size'] = 13

%run useful.py

# training data
perc_train = 0.8

In [7]:
# Keras works with numpy arrays: just use them from the start

TYPE=3
# data point size
L=8 
# span of each component
B=10
x = np.loadtxt(filename("data",L,TYPE), delimiter=' ')
y = np.loadtxt(filename("labels",L,TYPE), delimiter=' ')
y = y.astype("int")
N = len(x)
# average and std.dev.
x_mean = np.mean(x,axis=0)
x_std  = np.std(x,axis=0)
# dim. of a sample
L = len(x[0])
N_train = int(perc_train * N)
print(f'data: {N}\ntrain: {N_train}')
#plot_data(x,y)
#print("before rescaling:\nmean value=", x_mean,"\nstd. dev.=",x_std)
def Standardize(x,m,s):
    """
    rescale each component using its mean and standard deviation
    """
    N = len(x)
    # assuming len(m)=len(s)=len(x[0])
    mm,ss = np.tile(m,(N,1)), np.tile(s,(N,1))
    return (x-mm)/ss
x = Standardize(x,x_mean,x_std)
print("after rescaling:\nmean value=", x.mean(axis=0),"\nstd. dev.=",x.std(axis=0))

data: 2000
train: 1600
after rescaling:
mean value= [ 2.53633226e-15  9.27924404e-16 -1.44884105e-15 -2.38371996e-15
 -2.85274582e-15 -2.02196593e-15 -1.09010023e-15 -1.53632662e-15] 
std. dev.= [1. 1. 1. 1. 1. 1. 1. 1.]


# Split training - test

In [8]:
(x_train, y_train) = (x[0:N_train],y[0:N_train])
(x_valid, y_valid) = (x[N_train:],y[N_train:])
print("Train:",len(x_train),"\t Validation:",len(x_valid))

Train: 1600 	 Validation: 400


### Define the model

In [10]:
def build_model(hp):
    chosen_activation = "relu"
    chosen_dropout = 0.1
    #chosen_activation = hp.Choice("activation", ["relu", "tanh", "sigmoid", "elu"])
    #chosen_dropout = hp.Float("dropout_rate", min_value=0, max_value=0.2, step = 0.1)

    model = Sequential()
    # input layer
    model.add( Dense(L, input_shape = (L, ), 
            # Tune the activation function to use.
            activation= chosen_activation,
        )
    )
    # first hidden
    model.add( Dense(20,
            # Tune the activation function to use.
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))
    # second hidden
    model.add( Dense(20,
            # Tune the activation function to use.
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))
    # third hidden
    model.add( Dense(20,
            # Tune the activation function to use.
            activation=chosen_activation,
        )
    )
    model.add(Dropout(rate=chosen_dropout))
    # output layer
    model.add( Dense(1,
            # Tune the activation function to use.
            activation="sigmoid",
        )
    )

    model.compile(
        optimizer=Adam(learning_rate = hp.Float("lr", min_value=1e-6, max_value=1e-1, sampling="log")),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_2, built=True>

In [11]:
hp = keras_tuner.HyperParameters()


tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

tuner.search_space_summary()

Search space summary
Default search space size: 1
lr (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.1, 'step': None, 'sampling': 'log'}


In [12]:
tuner.search(x_train, y_train, epochs=400, validation_data=(x_valid, y_valid))

Trial 1 Complete [00h 00m 52s]
val_accuracy: 0.6362499892711639

Best val_accuracy So Far: 0.6362499892711639
Total elapsed time: 00h 00m 52s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
3.6724e-06        |5.5955e-05        |lr

Epoch 1/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5091 - loss: 0.6948 - val_accuracy: 0.4875 - val_loss: 0.6967
Epoch 2/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5079 - loss: 0.6969 - val_accuracy: 0.4900 - val_loss: 0.6967
Epoch 3/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4987 - loss: 0.6996 - val_accuracy: 0.4850 - val_loss: 0.6967
Epoch 4/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5224 - loss: 0.6921 - val_accuracy: 0.4850 - val_loss: 0.6966
Epoch 5/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5135 - loss: 0.6936 - val_accuracy: 0.4850 - val_loss: 0.6966
Epoch 6/400
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.5209 - loss: 0.6957 - val_accuracy

KeyboardInterrupt: 

In [7]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           180 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,113 (4.35 KB)

 Trainable params: 1,113 (4.35 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
activation: tanh
dropout_rate: 0.2
lr: 7.98129704930137e-05
Score: 0.5637499988079071

Trial 2 summary
Hyperparameters:
activation: tanh
dropout_rate: 0.1
lr: 2.0791985478621922e-05
Score: 0.5475000143051147

Trial 1 summary
Hyperparameters:
activation: elu
dropout_rate: 0.1
lr: 0.00032077017422736036
Score: 0.5437500178813934


In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
#model = build_model(best_hps[0])
# Fit with the entire dataset.
#x_all = np.concatenate((x_train, x_val))
#y_all = np.concatenate((y_train, y_val))
#model.fit(x=x_all, y=y_all, epochs=1)